In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers[torch] transformers
!pip install controlnet_aux
!pip install opencv-python
!pip install matplotlib
!pip install datasets

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.

In [3]:
!pip install -U diffusers

In [4]:
!pip install -U diffusers transformers torch torchvision accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [5]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl (848.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.4 MB/s eta 0:00:00


In [7]:
import cv2
import numpy as np
from PIL import Image
import os

# Function to extract edges from an image
def extract_edges(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(image, 100, 200)
    return edges

# Define dataset and output paths
dataset_path = "/content/drive/MyDrive/datafloor"  # Path to your dataset
output_path = "/content/drive/MyDrive/processed_dataset"  # Path to save processed images

# Create the output folder if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Preprocess all images in your dataset
for label_folder in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label_folder)
    if os.path.isdir(label_path):
        # Create a subfolder in the output directory for each label
        output_label_path = os.path.join(output_path, label_folder)
        os.makedirs(output_label_path, exist_ok=True)

        # Process each image in the label folder
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            edges = extract_edges(image_path)
            # Save the processed image
            output_image_path = os.path.join(output_label_path, image_name)
            cv2.imwrite(output_image_path, edges)

print("Dataset preprocessing complete!")

Dataset preprocessing complete!


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class FloorPlanDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Debug: Print the image folder being used
        print(f"Loading dataset from: {image_folder}")

        for label_folder in os.listdir(image_folder):
            label_path = os.path.join(image_folder, label_folder)
            if os.path.isdir(label_path):
                # Debug: Print the label folder being processed
                print(f"Processing label folder: {label_folder}")
                for image_name in os.listdir(label_path):
                    image_path = os.path.join(label_path, image_name)
                    self.image_paths.append(image_path)
                    self.labels.append(int(label_folder.split("_")[0]))  # Extract label from folder name

        # Debug: Print the total number of images loaded
        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import re  # Regular expressions for extracting numbers

class FloorPlanDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Debug: Print the image folder being used
        print(f"Loading dataset from: {image_folder}")

        for label_folder in os.listdir(image_folder):
            label_path = os.path.join(image_folder, label_folder)
            if os.path.isdir(label_path):
                # Debug: Print the label folder being processed
                print(f"Processing label folder: {label_folder}")

                # Extract the numeric part from the folder name (e.g., "3 rooms" -> 3)
                match = re.search(r'\d+', label_folder)  # Find the first number in the folder name
                if match:
                    label = int(match.group())  # Convert the numeric part to an integer
                else:
                    raise ValueError(f"Folder name '{label_folder}' does not contain a valid number.")

                # Process each image in the label folder
                for image_name in os.listdir(label_path):
                    image_path = os.path.join(label_path, image_name)
                    self.image_paths.append(image_path)
                    self.labels.append(label)  # Use the extracted label

        # Debug: Print the total number of images loaded
        print(f"Total images loaded: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [10]:
import os

# Path to the original dataset
original_dataset_path = "/content/drive/MyDrive/datafloor"

# List all folders in the original dataset
folders = [f for f in os.listdir(original_dataset_path) if os.path.isdir(os.path.join(original_dataset_path, f))]
print("Folders in original dataset:", folders)

Folders in original dataset: ['4 rooms', '3 rooms', '5 rooms']


In [11]:
# Path to the processed dataset
processed_dataset_path = "/content/drive/MyDrive/processed_dataset"

# List all folders in the processed dataset
folders = [f for f in os.listdir(processed_dataset_path) if os.path.isdir(os.path.join(processed_dataset_path, f))]
print("Folders in processed dataset:", folders)

Folders in processed dataset: ['4 rooms', '3 rooms', '5 rooms']


In [12]:
import cv2
import numpy as np
from PIL import Image
import os

# Function to extract edges from an image
def extract_edges(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(image, 100, 200)
    return edges

# Define dataset and output paths
dataset_path = "/content/drive/MyDrive/datafloor"  # Path to your original dataset
output_path = "/content/drive/MyDrive/processed_dataset"  # Path to save processed images

# Create the output folder if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Preprocess all images in your dataset
for label_folder in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label_folder)
    if os.path.isdir(label_path):
        # Create a subfolder in the output directory for each label
        output_label_path = os.path.join(output_path, label_folder)
        os.makedirs(output_label_path, exist_ok=True)

        # Process each image in the label folder
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            edges = extract_edges(image_path)
            # Save the processed image
            output_image_path = os.path.join(output_label_path, image_name)
            cv2.imwrite(output_image_path, edges)

print("Dataset preprocessing complete!")

Dataset preprocessing complete!


In [13]:
from torchvision import transforms

# Define transforms
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

# Create dataset
dataset = FloorPlanDataset(image_folder="/content/drive/MyDrive/processed_dataset", transform=transform)

# Create dataloader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Debug: Print a batch of data
for batch in dataloader:
    images, labels = batch
    print("Batch images shape:", images.shape)
    print("Batch labels:", labels)
    break

Loading dataset from: /content/drive/MyDrive/processed_dataset
Processing label folder: 4 rooms
Processing label folder: 3 rooms
Processing label folder: 5 rooms
Total images loaded: 510
Batch images shape: torch.Size([4, 3, 512, 512])
Batch labels: tensor([5, 3, 4, 4])


In [15]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
import torch

# Load ControlNet model
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny")

# Load Stable Diffusion model with ControlNet
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
).to("cuda")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
from torch.optim import AdamW
from tqdm import tqdm
from transformers import CLIPTextModel, CLIPTokenizer
import torch
import torch.nn.functional as F

# Load the tokenizer and text encoder for Stable Diffusion
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")

# Define optimizer
optimizer = AdamW(controlnet.parameters(), lr=1e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    controlnet.train()
    epoch_loss = 0

    for batch in tqdm(dataloader):
        control_images, labels = batch
        control_images = control_images.to("cuda")

        # Zero gradients
        optimizer.zero_grad()

        # Prepare text embeddings with proper padding
        text_inputs = tokenizer(
            [f"{label}-room floor plan" for label in labels],
            padding="max_length",
            max_length=tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt"
        )
        text_input_ids = text_inputs.input_ids.to("cuda")

        # Get text embeddings
        with torch.no_grad():
            text_embeddings = text_encoder(text_input_ids)[0]

        # Create appropriate noise sample - make sure dimensions match UNet expectations
        # Check the expected channel dimension of your UNet
        batch_size = control_images.shape[0]

        # Most SD models expect 4 channels for latents
        # The spatial dimensions should be 1/8 of the original image size
        # For 512×512 images, latents would be 64×64
        latent_height, latent_width = 64, 64
        noise = torch.randn(
            (batch_size, 4, latent_height, latent_width),
            device="cuda"
        )

        # Generate random timesteps
        timesteps = torch.randint(
            0, 1000, (batch_size,),
            device="cuda"
        ).long()

        # Make sure control images match expected input shape
        # If control_images aren't already transformed to match ControlNet's expected input
        # Add a transformation here if needed

        # Forward pass through ControlNet
        try:
            down_block_res_samples, mid_block_res_sample = controlnet(
                sample=noise,
                timestep=timesteps,
                encoder_hidden_states=text_embeddings,
                controlnet_cond=control_images,
                return_dict=False,
            )

            # Simple loss example (regularize outputs)
            loss = 0
            for res_sample in down_block_res_samples:
                loss += res_sample.pow(2).mean()
            loss += mid_block_res_sample.pow(2).mean()

            # Backward pass
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        except Exception as e:
            print(f"Error in batch: {e}")
            print(f"Text embeddings shape: {text_embeddings.shape}")
            print(f"Control images shape: {control_images.shape}")
            print(f"Noise shape: {noise.shape}")
            # Skip problematic batch
            continue

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch + 1} Loss: {avg_epoch_loss}")

# Save the fine-tuned model
controlnet.save_pretrained("/content/drive/MyDrive/fine_tuned_controlnet")

Epoch 1/3


  1%|          | 1/128 [00:00<00:16,  7.85it/s]

Error in batch: mat1 and mat2 shapes cannot be multiplied (308x512 and 768x320)
Text embeddings shape: torch.Size([4, 77, 512])
Control images shape: torch.Size([4, 3, 512, 512])
Noise shape: torch.Size([4, 4, 64, 64])


  2%|▏         | 2/128 [00:00<00:16,  7.57it/s]

Error in batch: mat1 and mat2 shapes cannot be multiplied (308x512 and 768x320)
Text embeddings shape: torch.Size([4, 77, 512])
Control images shape: torch.Size([4, 3, 512, 512])
Noise shape: torch.Size([4, 4, 64, 64])


...

Error in batch: mat1 and mat2 shapes cannot be multiplied (308x512 and 768x320)
Text embeddings shape: torch.Size([4, 77, 512])
Control images shape: torch.Size([4, 3, 512, 512])
Noise shape: torch.Size([4, 4, 64, 64])
Error in batch: mat1 and mat2 shapes cannot be multiplied (308x512 and 768x320)
Text embeddings shape: torch.Size([4, 77, 512])
Control images shape: torch.Size([4, 3, 512, 512])
Noise shape: torch.Size([4, 4, 64, 64])


 99%|█████████▉| 127/128 [00:16<00:00,  8.16it/s]

Error in batch: mat1 and mat2 shapes cannot be multiplied (308x512 and 768x320)
Text embeddings shape: torch.Size([4, 77, 512])
Control images shape: torch.Size([4, 3, 512, 512])
Noise shape: torch.Size([4, 4, 64, 64])
Error in batch: mat1 and mat2 shapes cannot be multiplied (308x512 and 768x320)
Text embeddings shape: torch.Size([4, 77, 512])
Control images shape: torch.Size([4, 3, 512, 512])
Noise shape: torch.Size([4, 4, 64, 64])


100%|██████████| 128/128 [00:17<00:00,  7.52it/s]


Error in batch: mat1 and mat2 shapes cannot be multiplied (154x512 and 768x320)
Text embeddings shape: torch.Size([2, 77, 512])
Control images shape: torch.Size([2, 3, 512, 512])
Noise shape: torch.Size([2, 4, 64, 64])
Epoch 3 Loss: 0.0


In [26]:
# Set environment variable for memory management
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Then proceed with model loading
torch.cuda.empty_cache()

# Load the fine-tuned ControlNet model in half precision
fine_tuned_controlnet = ControlNetModel.from_pretrained(
    "/content/drive/MyDrive/fine_tuned_controlnet",
    torch_dtype=torch.float16
).to("cuda")

# Remaining code...

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 40081 has 14.74 GiB memory in use. Of the allocated memory 14.49 GiB is allocated by PyTorch, and 125.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [25]:
# Load the fine-tuned ControlNet model in half precision
fine_tuned_controlnet = ControlNetModel.from_pretrained(
    "/content/drive/MyDrive/fine_tuned_controlnet",
    torch_dtype=torch.float16
).to("cuda")

# Load Stable Diffusion with the fine-tuned ControlNet (also in half precision)
fine_tuned_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=fine_tuned_controlnet,
    torch_dtype=torch.float16
).to("cuda")

# Enable memory efficient attention
fine_tuned_pipe.enable_attention_slicing()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 40081 has 14.73 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 140.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [24]:
# First clear cache
torch.cuda.empty_cache()

# Load the text encoder first
text_encoder = CLIPTextModel.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="text_encoder",
    torch_dtype=torch.float16
).to("cuda")

# Load the tokenizer (doesn't use GPU memory)
tokenizer = CLIPTokenizer.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="tokenizer"
)

# Load VAE
vae = AutoencoderKL.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="vae",
    torch_dtype=torch.float16
).to("cuda")

# Load the control net model
fine_tuned_controlnet = ControlNetModel.from_pretrained(
    "/content/drive/MyDrive/fine_tuned_controlnet",
    torch_dtype=torch.float16
).to("cuda")

# Load the UNet
unet = UNet2DConditionModel.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="unet",
    torch_dtype=torch.float16
).to("cuda")

# Create the pipeline with all components
fine_tuned_pipe = StableDiffusionControlNetPipeline(
    vae=vae,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    unet=unet,
    controlnet=fine_tuned_controlnet,
    scheduler=DDIMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler"),
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False
)

# Memory optimizations
fine_tuned_pipe.enable_attention_slicing(slice_size=1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 52.12 MiB is free. Process 40081 has 14.69 GiB memory in use. Of the allocated memory 14.39 GiB is allocated by PyTorch, and 179.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
# Load the fine-tuned ControlNet model in float32 (default)
fine_tuned_controlnet = ControlNetModel.from_pretrained(
    "/content/drive/MyDrive/fine_tuned_controlnet"
).to("cuda")

# Load Stable Diffusion with the fine-tuned ControlNet in float32
fine_tuned_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=fine_tuned_controlnet,
    # Remove torch_dtype parameter to use float32
).to("cuda")

# Generate a floor plan
prompt = "a modern 3-room floor plan"
control_image = Image.open("/content/drive/MyDrive/processed_dataset/3 rooms/Cat10_1.jpg")
control_image = control_image.resize((512, 512))
output_image = fine_tuned_pipe(prompt, control_image, num_inference_steps=50).images[0]

# Display the generated image
output_image.show()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 114.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 52.12 MiB is free. Process 40081 has 14.69 GiB memory in use. Of the allocated memory 14.39 GiB is allocated by PyTorch, and 179.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
# Load the fine-tuned ControlNet model
fine_tuned_controlnet = ControlNetModel.from_pretrained("/content/drive/MyDrive/fine_tuned_controlnet")

# Load Stable Diffusion with the fine-tuned ControlNet
fine_tuned_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=fine_tuned_controlnet, torch_dtype=torch.float16
).to("cuda")

# Generate a floor plan
prompt = "a modern 3-room floor plan"
control_image = Image.open("/content/drive/MyDrive/processed_dataset/3 rooms/Cat10_1.jpg")  # Use a control image
output_image = fine_tuned_pipe(prompt, control_image, num_inference_steps=50).images[0]

# Display the generated image
output_image.show()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 must have the same dtype, but got Half and Float